In [1]:
from premise import *
import pandas as pd

In [2]:
import bw2data
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [3]:
ndb = NewDatabase(
    scenarios=[
        {"model":"remind", "pathway":"SSP1-Base", "year":2050},
        {"model":"remind", "pathway":"SSP1-NDC", "year":2050},
        {"model":"remind", "pathway":"SSP1-NPi", "year":2050},
        {"model":"remind", "pathway":"SSP1-PkBudg1150", "year":2050},
        {"model":"remind", "pathway":"SSP1-PkBudg500", "year":2050},
        {"model":"remind", "pathway":"SSP2-Base", "year":2050},
        {"model":"remind", "pathway":"SSP2-NDC", "year":2050},
        {"model":"remind", "pathway":"SSP2-NPi", "year":2050},
        {"model":"remind", "pathway":"SSP2-PkBudg1150", "year":2050},
        {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
        {"model":"remind", "pathway":"SSP5-Base", "year":2050},
        {"model":"remind", "pathway":"SSP5-NDC", "year":2050},
        {"model":"remind", "pathway":"SSP5-NPi", "year":2050},
        {"model":"remind", "pathway":"SSP5-PkBudg1150", "year":2050},
        {"model":"remind", "pathway":"SSP5-PkBudg500", "year":2050},
        {"model":"image", "pathway":"SSP1-Base", "year":2020},
        {"model":"image", "pathway":"SSP2-Base", "year":2020},
        {"model":"image", "pathway":"SSP2-RCP26", "year":2020},
        {"model":"image", "pathway":"SSP2-RCP19", "year":2020},
        {"model":"tiam-ucl", "pathway":"SSP2-Base", "year":2100},
        {"model":"tiam-ucl", "pathway":"SSP2-RCP45", "year":2100},
        {"model":"tiam-ucl", "pathway":"SSP2-RCP26", "year":2100},
        {"model":"tiam-ucl", "pathway":"SSP2-RCP19", "year":2100},
    ],
    source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    biosphere_name="biosphere3",
    keep_imports_uncertainty=True
)

premise requires the name of your biosphere database.
Please enter the name of your biosphere database as it appears in your project.
Databases dictionary with 12 object(s):
	Swiss_trying with update external for SSP2-PkBudg1150 2030,2050
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ecoinvent-3.10-cutoff - remind - SSP2-PkBudg1150
	ei_cutoff_3.10_tiam-ucl_SSP2-Base_2050 2024-10-12
	ei_cutoff_3.10_tiam-ucl_SSP2-Base_2100 2024-10-10
	stem 2020
	stem 2050
	superstructure
	test1
	test2
	test3
premise v.(2, 1, 3)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             

In [4]:
import pandas as pd

# Load the Excel file to check all sheet names
file_path = 'mapping_overview.xlsx'
excel_file = pd.ExcelFile(file_path)

# Display all sheet names
sheet_names = excel_file.sheet_names

# Load data from each sheet into a dictionary of dataframes
dfs = {sheet: pd.read_excel(file_path, sheet_name=sheet) for sheet in sheet_names}

# Concatenate all the dataframes into one, adding a column to indicate the source sheet
combined_df = pd.concat([df.assign(Sheet=sheet) for sheet, df in dfs.items()], ignore_index=True)


In [7]:
combined_df = combined_df.loc[combined_df["Variable Type"]=="Production volume"]

In [8]:
large_data = pd.DataFrame()
for scenario in ndb.scenarios:
    model = scenario["model"]
    pathway = scenario["pathway"]
    print(model, pathway)
    for i, row in combined_df.loc[combined_df["IAM Model"]==model].groupby(
        ["Sheet", "Variable"]
    ):
        var, _, _, _, sector = row.values.tolist()[0]

        data = None
        try:
            if var in (
                "CO2", "gdp", "population", "GMST"
            ):
                data = scenario["iam data"].other_vars.sel(variables=var)
            else:
                data = scenario["iam data"].production_volumes.sel(variables=var)
        except:
            print("ERROR", var)
        
        if data is not None:
            long_data = data.to_dataframe("val").reset_index()
            long_data["sector"] = sector
            long_data["model"] = model
            long_data["scenario"] = pathway
            
            long_data = long_data.loc[long_data["year"] <= 2100]
            
            large_data = pd.concat([large_data, long_data])
        
                

remind SSP1-Base
ERROR cement - co2
ERROR cement - co2
ERROR steel - co2
ERROR cement - co2
ERROR steel - co2
ERROR cement - co2
ERROR Gas OC
ERROR petrol, synthetic, from electrolysis
ERROR petrol, synthetic, from coal, with CCS
ERROR petrol, synthetic, from coal
remind SSP1-NDC
ERROR cement - co2
ERROR cement - co2
ERROR steel - co2
ERROR cement - co2
ERROR steel - co2
ERROR cement - co2
ERROR Gas OC
ERROR petrol, synthetic, from electrolysis
ERROR petrol, synthetic, from coal, with CCS
ERROR petrol, synthetic, from coal
remind SSP1-NPi
ERROR cement - co2
ERROR cement - co2
ERROR steel - co2
ERROR cement - co2
ERROR steel - co2
ERROR cement - co2
ERROR Gas OC
ERROR petrol, synthetic, from electrolysis
ERROR petrol, synthetic, from coal, with CCS
ERROR petrol, synthetic, from coal
remind SSP1-PkBudg1150
ERROR cement - co2
ERROR cement - co2
ERROR steel - co2
ERROR cement - co2
ERROR steel - co2
ERROR cement - co2
ERROR Gas OC
ERROR petrol, synthetic, from electrolysis
ERROR petrol, sy

In [9]:
len(large_data)

677820

In [10]:
large_data.loc[large_data["variables"]=="population", "sector"] = "Population"
large_data.loc[large_data["variables"]=="CO2", "sector"] = "Carbon Dioxide emissions"
large_data.loc[large_data["variables"]=="gdp", "sector"] = "Gross Domestic Product"
large_data.loc[large_data["variables"]=="GMST", "sector"] = "GMST increase"
large_data.loc[large_data["sector"]=="Transport Railfreight", "sector"] = "Freight (rail)"
large_data.loc[large_data["sector"]=="Transport Roadfreight", "sector"] = "Freight (road)"
large_data.loc[large_data["sector"]=="Transport Passenger Cars", "sector"] = "Passenger (car)"
large_data.loc[large_data["sector"]=="Transport Bus", "sector"] = "Passenger (bus)"
large_data.loc[large_data["sector"]=="Transport Two Wheelers", "sector"] = "Passenger (two-wheeler)"

large_data.loc[
    (large_data["variables"].str.contains("diesel"))
    &(~large_data["variables"].str.contains("truck"))
    &(~large_data["variables"].str.contains("train"))
    &(~large_data["variables"].str.contains("bus"))
    &(~large_data["variables"].str.contains("car"))
    , "sector"] = "Diesel"

large_data.loc[large_data["variables"].str.contains("ethanol"), "sector"] = "Gasoline"
large_data.loc[
    (large_data["variables"].str.contains("gasoline"))
    &(~large_data["variables"].str.contains("truck"))
    &(~large_data["variables"].str.contains("train"))
    &(~large_data["variables"].str.contains("bus"))
    &(~large_data["variables"].str.contains("car"))
    &(~large_data["variables"].str.contains("two-wheeler"))
    , "sector"] = "Gasoline"

large_data.loc[large_data["variables"].str.contains("liquefied"), "sector"] = "LPG"
large_data.loc[large_data["variables"].str.contains("kerosene"), "sector"] = "Kerosene"
large_data.loc[large_data["variables"].str.contains("hydrogen"), "sector"] = "Hydrogen"

large_data.loc[large_data["variables"]=="natural gas", "sector"] = "Gas"
large_data.loc[large_data["variables"]=="biomethane", "sector"] = "Gas"
large_data.loc[large_data["variables"]=="heavy fuel oil", "sector"] = "Oil"

large_data = large_data.replace("truck, ", "", regex=True)
large_data = large_data.replace("train, ", "", regex=True)
large_data = large_data.replace("bus, ", "", regex=True)
large_data = large_data.replace("passenger car, ", "", regex=True)
large_data = large_data.replace("two-wheeler, ", "", regex=True)

large_data = large_data.replace(", mini", "", regex=True)
large_data = large_data.replace(", medium SUV", "", regex=True)
large_data = large_data.replace(", large SUV", "", regex=True)
large_data = large_data.replace(", medium", "", regex=True)
large_data = large_data.replace(", van", "", regex=True)
large_data = large_data.replace(", large", "", regex=True)
large_data = large_data.replace(", small", "", regex=True)

large_data = large_data.replace(", 3.5t", "", regex=True)
large_data = large_data.replace(", 7.5t", "", regex=True)
large_data = large_data.replace(", 18t", "", regex=True)
large_data = large_data.replace(", 26t", "", regex=True)
large_data = large_data.replace(", 40t", "", regex=True)

large_data = large_data.replace(", energy allocation", "", regex=True)
large_data = large_data.replace("kerosene, ", "", regex=True)
large_data = large_data.replace(", PEM", "", regex=True)
large_data = large_data.replace("hydrogen, ", "", regex=True)
large_data = large_data.replace("liquefied petroleum gas, ", "LPG", regex=True)
large_data = large_data.replace("steel - ", "", regex=True)

large_data = large_data.groupby([
    "region", "year", "variables", "sector", "model", "scenario"
]).sum().reset_index()


In [11]:
large_data.loc[
    (large_data["model"].isin(["image", "remind"]))
    &(large_data["variables"]=="CO2"),
    "val"
] *= 1000000
large_data.loc[
    (large_data["model"]=="tiam-ucl")
    &(large_data["variables"]=="CO2"),
    "val"
] *= 1000
large_data.loc[
    (large_data["model"]=="tiam-ucl")
    &(large_data["variables"]=="gdp"),
    "val"
] *= 10000

large_data.loc[
    (large_data["model"]=="tiam-ucl")
    &(large_data["variables"].isin(['biomass crops - purpose grown', 'biomass - residual'])),
    "val"
] /= 1000

large_data.loc[
    (large_data["model"]=="remind")
    &(large_data["variables"].isin(['biomass crops - purpose grown', 'biomass - residual'])),
    "val"
] *= 10

large_data.loc[
    (large_data["model"]=="tiam-ucl")
    &(large_data["sector"]=="Electricity"),
    "val"
] /= 1000



In [12]:
large_data = large_data.loc[large_data["val"] != 0]

In [13]:
large_data.to_csv("../data/structured_data.csv")

In [132]:
large_data.loc[
    (large_data["model"]=="image")
    &(large_data["variables"].str.contains("cement"))
    &(large_data["region"]=="World")
]

,region,year,variables,val,sector,model,scenario
390,World,2005,"cement, dry feed rotary kiln",0.0,Cement,image,SSP2-Base
391,World,2010,"cement, dry feed rotary kiln",0.0,Cement,image,SSP2-Base
392,World,2015,"cement, dry feed rotary kiln",0.0,Cement,image,SSP2-Base
393,World,2020,"cement, dry feed rotary kiln",0.0,Cement,image,SSP2-Base
394,World,2025,"cement, dry feed rotary kiln",0.0,Cement,image,SSP2-Base
...,...,...,...,...,...,...,...
400,World,2060,"cement, dry feed rotary kiln, efficient, with ...",0.0,Cement,image,SSP2-Base
401,World,2070,"cement, dry feed rotary kiln, efficient, with ...",0.0,Cement,image,SSP2-Base
402,World,2080,"cement, dry feed rotary kiln, efficient, with ...",0.0,Cement,image,SSP2-Base
403,World,2090,"cement, dry feed rotary kiln, efficient, with ...",0.0,Cement,image,SSP2-Base
